<a href="https://colab.research.google.com/github/roy392/AiPinter-v2.0/blob/master/_4_Roy_%20Fernando_HandsOn_ReutersNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classifying newswires: a multiclass classification example**
The Reuters dataset
You’ll work with the Reuters dataset, a set of short newswires and their topics, published by Reuters in 1986. It’s a simple, widely used toy dataset for text classification. There are 46 different topics; some topics are more represented than others, but each topic has at least 10 examples in the training set.

Preparing the dataset

In [1]:
#Import dataset
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data( num_words=10000)

2110848/2110848 [==============================] - 0s 0us/step


In [ ]:
#Check the train data and train labels, Write your comment/review on the those train data n train label
#Write your code here, put your comment


**Decoding newswires back to text**

In [ ]:
word_index = reuters.get_word_index() 
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) 
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

**Preparing the data**

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension)) 
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

x_train = vectorize_sequences(train_data) 
x_test = vectorize_sequences(test_data)

In [ ]:
x_train.shape

(8982, 10000)

To vectorize the labels, there are two possibilities: you can cast the label list as an integer tensor, or you can use one-hot encoding. One-hot encoding is a widely used format for categorical data, also called categorical encoding. For a more detailed explanation of one-hot encoding, see section 6.1. In this case, one-hot encoding of the labels consists of embedding each label as an all-zero vector with a 1 in the place of the label index. Here’s an example:

In [ ]:
def to_one_hot(labels, dimension=46):
  results = np.zeros((len(labels), dimension)) 
  for i, label in enumerate(labels):
    results[i, label] = 1.
  return results
one_hot_train_labels = to_one_hot(train_labels) 
one_hot_test_labels = to_one_hot(test_labels)

In [ ]:
#Check one_hot_train_labels
#Code your write here
one_hot_train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Note that there is a built-in way to do this in Keras, which you’ve already seen in action in the MNIST example:

In [ ]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels) 
one_hot_test_labels = to_categorical(test_labels)

In [ ]:
#Check one_hot_train_labels
#Code your write here

**Building your network**


In the previous example, you used 16-dimensional intermediate layers, but a 16-dimensional space may be too limited to learn to separate 46 different classes: such small layers may act as information bottlenecks, permanently dropping relevant information.
For this reason you’ll use larger layers. Let’s go with 64 units.

In [ ]:
#Model definition
from keras import models 
from keras import layers

model = models.Sequential() 
model.add(layers.Dense(64, activation='relu', input_shape=(10000,))) 
model.add(layers.Dense(64, activation='relu')) 
model.add(layers.Dense(46, activation='softmax'))

In [ ]:
x_val = x_train[:10000] 
partial_x_train = x_train[10000:]
y_val = one_hot_train_labels[:10000] 
partial_y_train =one_hot_train_labels[10000:]

In [ ]:
partial_x_train.shape

(0, 10000)

**Compiling the model**

In [ ]:
model.compile(optimizer='rmsprop',
  loss='categorical_crossentropy', metrics=['accuracy'])

**Validating The Approach**
Let’s set apart 1,000 samples in the training data to use as a validation set.

Training your model

In [ ]:
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20


ValueError: ignored

Plotting the training and validation loss

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss'] 
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.xlabel('Epochs') 
plt.ylabel('Loss') 
plt.legend()
plt.show()

Plotting the training and validation accuracy

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
plt.clf() 
#Clears the figure 
acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.xlabel('Epochs') 
plt.ylabel('Loss') 
plt.legend()

plt.show()

The network begins to overfit after nine epochs. Let’s train a new network from scratch for nine epochs and then evaluate it on the test set.

**Retraining a model from scratch**


In [ ]:
model = models.Sequential() 
model.add(layers.Dense(64, activation='relu', input_shape=(10000,))) 
model.add(layers.Dense(64, activation='relu')) 
model.add(layers.Dense(46, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(partial_x_train, partial_y_train, epochs=9, batch_size=512, validation_data=(x_val, y_val)) 
results = model.evaluate(x_test, one_hot_test_labels)

Using a trained network to generate predictions on new data
You can verify that the predict method of the model instance returns a probability distribution over all 46 topics. Let’s generate topic predictions for all of the test data.

In [ ]:
predictions=model.predict(x_test)

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
#Menghitung akurasi hasil prediksi model 
from tensorflow.keras import metrics
m = metrics.Accuracy()
m.update_state( predictions.argmax(axis=1), one_hot_test_labels.argmax(axis=1))
m.result().numpy()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
print(classification_report(predictions.argmax(axis=1)), one_hot_test_labels.argmax(axis=1))#, target_names=class_names))
print(confusion_matrix(predictions.argmax(axis=1), one_hot_test_labels.argmax(axis=1)))

**Further experiments**

Try using larger or smaller layers: 32 units, 128 units, and so on.

1.   You used two hidden layers. 
2.   Now try using a single hidden layer, or three hidden layers.



In [ ]:
#HANDS ON...
#Write your code here